# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Libraries and Import

In [ ]:
%%capture
!pip install clodsa
!pip install -q -U albumentations

In [ ]:
!pip uninstall imgaug

In [ ]:
%%capture
!pip install git+https://github.com/aleju/imgaug.git

In [ ]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import random
import cv2
import albumentations as A
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import os 

# CLoDSA

## Dropout

In [ ]:
def apply_dropout(input_path, output_path, percentages):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  for percentage in percentages:
    dropout = createTechnique("dropout", {"percentage" : percentage})
    augmentor.addTransformer(transformer(dropout))
  
  augmentor.applyAugmentation()
  print("Dropout results were saved given directory.")

In [ ]:
# 0.05, 0,1 should be in seperate folders
apply_dropout("/content/drive/MyDrive/AI_Projects/STARE/png_data/train",
               "/content/drive/MyDrive/AI_Projects/STARE/png_data/dropout",
               [0.1])

## Gamma Correction

In [ ]:
def apply_gamma_correction(input_path, output_path, gammas):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  for gamma in gammas:
    gamma_t = createTechnique("gamma", {"gamma" : gamma})
    augmentor.addTransformer(transformer(gamma_t))
  
  augmentor.applyAugmentation()
  print("Gamma correction results were saved given directory.")

In [ ]:
#1.5, 2 should be in seperate folders
apply_gamma_correction("/content/drive/MyDrive/AI_Projects/STARE/png_data/train",
               "/content/drive/MyDrive/AI_Projects/STARE/png_data/gamma_correction05",
               [0.5])

## White Noise

In [ ]:
def apply_white_noise(input_path, output_path, sd):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  for sigma in sd:
    white_noise = createTechnique("gaussian_noise", {"mean" : 0,"sigma":sigma})
    augmentor.addTransformer(transformer(white_noise))
  
  augmentor.applyAugmentation()
  print("White noise results were saved given directory.")

In [ ]:
apply_white_noise("/content/drive/MyDrive/AI_Projects/STARE/png_data/train",
               "/content/drive/MyDrive/AI_Projects/STARE/png_data/white_noise",
               [10])

## Equalize Histogram

In [ ]:
 def apply_eqhisto(input_path, output_path):
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)
  equalize = createTechnique("equalize_histogram",{})
  augmentor.addTransformer(transformer(equalize)) 
  augmentor.applyAugmentation()

  print("Equalize histogram results were saved given directory.")

In [ ]:
apply_eqhisto("/content/drive/MyDrive/AI_Projects/STARE/png_data/train", 
              "/content/drive/MyDrive/AI_Projects/STARE/png_data/eq_hist")

## Blurring

In [ ]:
def aug_blurring(input_path, output_path, blurr:list):
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})
  transformer = transformerGenerator(PROBLEM)

  for ker in blurr:
    blur = createTechnique("blurring", {"kernel" : ker})
    augmentor.addTransformer(transformer(blur))
    print("Blurring for kernel = {} is done".format(ker))
  
  augmentor.applyAugmentation()
  print("Augmentation results were saved given directory.")

In [ ]:
#3, 5 should be in seperate folders
aug_blurring("/content/drive/MyDrive/AI_Projects/STARE/png_data/train",
             "/content/drive/MyDrive/AI_Projects/STARE/png_data/blurring5", [5])

## Elastic Deformation

In [ ]:
def apply_elastic_deformation(input_path, output_path, alpha = 5, sigma = 0.05):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("elastic", {"alpha" : alpha, "sigma" : sigma})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Elastic deformation results were saved given directory.")

In [ ]:
apply_elastic_deformation('/content/drive/MyDrive/AI_Projects/STARE/png_data/train',
                          '/content/drive/MyDrive/AI_Projects/STARE/png_data/elastic')

## Flipping

In [ ]:
def apply_flipping(input_path, output_path, flip):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("flip", {"flip" : flip})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Flipping results were saved given directory.")

In [ ]:
apply_flipping('/content/drive/MyDrive/AI_Projects/STARE/png_data/train',
                '/content/drive/MyDrive/AI_Projects/STARE/png_data/flipping1', 1)

## Shearing

In [ ]:
def apply_shearing(input_path, output_path, a = 0.5):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("shearing", {"a" : a})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Shearing results were saved given directory.")

In [ ]:
apply_shearing('/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/train',
                '/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/shearing',a = 0.3)

## Sharpen

In [ ]:
def apply_sharpen(input_path, output_path):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("sharpen", {})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Sharping results were saved given directory.")

In [ ]:
apply_sharpen('/content/drive/MyDrive/AI_Projects/STARE/png_data/train',
                '/content/drive/MyDrive/AI_Projects/STARE/png_data/sharpen')

## Raise Saturation

In [ ]:
def apply_raise_satur(input_path, output_path, power):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("raise_saturation", {"power" : power})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Raise saturation results were saved given directory.")

In [ ]:
apply_raise_satur('/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/train',
                '/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/saturation4', 0.5)

# IMGAUG

## Rotation

In [ ]:
input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train'
output_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/deneme8'

In [ ]:
def rotation(degree, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.Rotate((degree)),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'rotat'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'rotat'+ '_' + str(indx) + '.png', i)

  print("Rotation results were saved given directory.")

In [ ]:
rotation(30,input_path,output_path,20)

## JPEG Compression

In [ ]:
import imageio
import imgaug as ia
import os 

def apply_jpeg_compression(input_path, output_path, degrees):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - degrees -> this should be a list that includes the degrees that you want to compress images and labels (must be between 0 and 100).
  '''

  images = []
  labels = []

  for img_path in range(20):
    img = imageio.imread(input_path + 'images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + 'labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path) 

  for degree in degrees:
    aug = ia.augmenters.JpegCompression(compression=degree)
    
    images_aug = aug.augment_images(images=images)
    for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + 'images/' + str(degree) + '_' + str(indx) + '.png', i)

    labels_aug = aug.augment_images(images=labels)
    for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + 'labels/' + str(degree) + '_' + str(indx) + '.png', i)

  print("JPEG Compression results were saved given directory.")

In [ ]:
# 25,50,75 should be in seperate folders

apply_jpeg_compression("/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/train/",
                       "/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/jpeg_compression25/",
                        [25])

## Zoom In / Out

In [ ]:
input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train'
output_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/deneme7'

In [ ]:
def zoom(zoom_amount, input_path, output_path, image_count):
  images = []
  labels = []
  ia.seed(1)
  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.Affine(
              scale={"x": (zoom_amount), "y": (zoom_amount)}
          ),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'zoom'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'zoom'+ '_' + str(indx) + '.png', i)

  print("Zoom results were saved given directory.")

In [ ]:
zoom(0.8, input_path, output_path, 20)

## Shear X/Y

In [ ]:
input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train'
output_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/deneme6'

In [ ]:
def shearX(shear_amount, input_path, output_path, image_count):
  images = []
  labels = []
  ia.seed(1)
  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.ShearX((shear_amount)),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'shear'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'shear'+ '_' + str(indx) + '.png', i)

  print("Shear results were saved given directory.")

In [ ]:
def shearY(shear_amount, input_path, output_path, image_count):
  images = []
  labels = []
  ia.seed(1)
  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.ShearY((shear_amount)),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'shear'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'shear'+ '_' + str(indx) + '.png', i)

  print("Shear results were saved given directory.")

In [ ]:
shearX(20, input_path, output_path, 20)

## Contrast

In [ ]:
%%capture
!pip install imagecorruptions

In [ ]:
input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train'
output_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/deneme5'

In [ ]:
def contrast(severity, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
    [

        iaa.imgcorruptlike.Contrast(severity=severity),

    ]
  )

  images_aug = seq(images=images)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'contrast'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels):
      imageio.imwrite(output_path + '/labels/'  + 'contrast'+ '_' + str(indx) + '.png', i)

  print("Contrast results were saved given directory.")

In [ ]:
contrast(2,input_path, output_path, image_count= 20)


## Shift X Y

In [ ]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
import os

input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train/'
output_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/aug/'

shift_amount = -100 #pixel
image_count  = 20

In [ ]:
def shiftX(shift_amount, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + 'images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + 'labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
    [

        iaa.TranslateX(px=(shift_amount))

    ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + 'images/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + 'labels/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  print("Shift results were saved given directory.")

In [ ]:
def shiftY(shift_amount, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + 'images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + 'labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
    [

        iaa.TranslateY(px=(shift_amount))

    ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + 'images/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + 'labels/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  print("Shift results were saved given directory.")

In [ ]:
shiftX(shift_amount, input_path, output_path, image_count)

# Albumentations



1.   Create your main folder in your path
2.   Change the paths inside the code properly
3.   Choose the method from [here](https://albumentations.ai/docs/examples/example_kaggle_salt/)
4.   Change the part between hashtags (####) with this method
5.   Output will be in your main path



In [ ]:
import os
import cv2

output_folder_name = 'deneme'

main_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data'
input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train'

original_height, original_width = 608, 704

In [ ]:
print(original_height)
print(original_width)

In [ ]:
def albumentation(output_folder_name, main_path, original_height, original_width, input_path):

  '''
    - output_folder_name : you should give just the name of the output folder, it will be created by function
    - main_path : the folder that output folder will be created and results will be saved
    - input_path : the folder that includes images and labels in seperate folders
  '''

  os.mkdir(main_path + '/'+ output_folder_name)
  os.mkdir(main_path + '/'+ output_folder_name +'/images')
  os.mkdir(main_path + '/'+ output_folder_name +'/labels')

  for img in sorted(os.listdir(input_path + '/images')):

    image = cv2.imread(input_path +'/images/' + img, 0)
    mask  = cv2.imread(input_path +'/labels/' + img, 0)
    
    ##############################################################
    aug = A.Compose([
      A.OneOf([
          A.RandomSizedCrop(min_max_height=(50, 101), height=original_height, width=original_width, p=0.5),
          A.PadIfNeeded(min_height=original_height, min_width=original_width, p=0.5)
      ], p=1),    
      A.VerticalFlip(p=0.5),              
      A.RandomRotate90(p=0.5),
      A.OneOf([
          A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
          A.GridDistortion(p=0.5),
          A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)                  
          ], p=0.8),
      A.CLAHE(p=0.8),
      A.RandomBrightnessContrast(p=0.8),    
      A.RandomGamma(p=0.8)])
    ##############################################################

    augmented = aug(image=image, mask=mask)

    image = augmented['image']
    mask = augmented['mask']

    cv2.imwrite(main_path +'/'+ output_folder_name +'/images/' + img, image)
    cv2.imwrite(main_path +'/' + output_folder_name +'/labels/' + img, mask)

In [ ]:
albumentation(output_folder_name, main_path, original_height, original_width, input_path)

# Append One Directory to Another

In [ ]:
import os, shutil

def merge_augmentations(augment_dir, output_dir, list_of_aug_files):
  '''
    - augment_dir         (string)   : The path which has subfolders that are augmentation folders.
    - output_dir          (string)   : The path you want to put all augmentations.
    - list_of_aug_files   (list)     : List of names which contains augmentations.
  '''
  
  os.mkdir(output_dir + '/images')
  os.mkdir(output_dir + '/labels')
  

  for folder in list_of_aug_files:
    if folder != 'train':
      for file in sorted(os.listdir(augment_dir + '/' + folder + '/images')):
        shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + folder + '_' + file.split("_")[-1].split('.')[0] + '.png')
        shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + folder + '_' + file.split("_")[-1].split('.')[0] + '.png')
        #shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + file)
        #shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + file)
    else:
      for file in sorted(os.listdir(augment_dir + '/' + folder + '/images')):
        shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + file.split("_")[-1].split('.')[0] + '.png')
        shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + file.split("_")[-1].split('.')[0] + '.png')
        #shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + file)
        #shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + file)

    print(folder + ' folder has been merged...')
    print('Number of images in output: ' + str(len(os.listdir(output_dir + '/images'))))
  
  print('Merging is done successfully!')

In [ ]:
merge_augmentations('/content/drive/MyDrive/AI_Projects/STARE/png_data',
                    '/content/drive/MyDrive/AI_Projects/STARE/combinations_png/B', 
                    ['train', 
                     'flipping0', 
                     'flipping1', 
                     'rotat30', 
                     'rotat60', 
                     'rotat90', 
                     'rotat120', 
                     'rotat150', 
                     'rotat180', 
                     'rotat210', 
                     'rotat240', 
                     'rotat270', 
                     'rotat300', 
                     'rotat330', 
                     'flipping0_30','flipping0_60',
                     'flipping0_90','flipping0_120',
                     'flipping0_150',
                     'flipping0_180',
                     'flipping0_210','flipping0_240',
                     'flipping0_270',
                     'flipping0_300','flipping0_330',
                     'flipping1_30','flipping1_60',
                     'flipping1_90','flipping1_120',
                     'flipping1_150',
                     'flipping1_180',
                     'flipping1_210','flipping1_240',
                     'flipping1_270',
                     'flipping1_300','flipping1_330',
                     'zoom_out08',
                     'zoom_out08_30', 'zoom_out08_60',
                     'zoom_out08_90',
                     'zoom_out08_120',
                     'zoom_out08_180','zoom_out08_150',
                     'zoom_out08_210','zoom_out08_240',
                     'zoom_out08_270',
                     'zoom_out08_300',
                     'zoom_out08_330',
                     'white_noise','white_noise90','white_noise180','white_noise270',
                     'elastic',
                     'shifty_100','shifty_-100','shiftx_100','shiftx_-100',
                     'shifty_100_90','shifty_100_180','shifty_100_270',
                     'shifty_-100_90','shifty_-100_180','shifty_-100_270',
                     'shiftx_100_90','shiftx_100_180','shiftx_100_270',
                     'shiftx_-100_90','shiftx_-100_180','shiftx_-100_270',
                     'gamma_correction05','gamma_correction05_30','gamma_correction05_60',
                     'gamma_correction05_90','gamma_correction05_120','gamma_correction05_150',
                     'gamma_correction05_180','gamma_correction05_210','gamma_correction05_240',
                     'gamma_correction05_270','gamma_correction05_300','gamma_correction05_330',
                     'album_grid_dist','album_heavy', 'album_medium', 'album_optical','album_random_crop',
                     'album_grid_dist90','album_grid_dist180','album_grid_dist270',
                     'album_heavy90','album_heavy180','album_heavy270','album_medium90','album_medium180','album_medium270',
                     'album_optical90', 'album_optical180', 'album_optical270','album_random_crop2','album_random_crop3','album_random_crop4',
                     'blurring3', 'blurring5','dropout','eq_hist', 'sharpen'])

train folder has been merged...
Number of images in output: 20
flipping0 folder has been merged...
Number of images in output: 40
flipping1 folder has been merged...
Number of images in output: 60
rotat30 folder has been merged...
Number of images in output: 80
rotat60 folder has been merged...
Number of images in output: 100
rotat90 folder has been merged...
Number of images in output: 120
rotat120 folder has been merged...
Number of images in output: 140
rotat150 folder has been merged...
Number of images in output: 160
rotat180 folder has been merged...
Number of images in output: 180
rotat210 folder has been merged...
Number of images in output: 200
rotat240 folder has been merged...
Number of images in output: 220
rotat270 folder has been merged...
Number of images in output: 240
rotat300 folder has been merged...
Number of images in output: 260
rotat330 folder has been merged...
Number of images in output: 280
flipping0_30 folder has been merged...
Number of images in output: 300

# Experimental Style Transfer Part

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install -q 'scipy<=1.2.1'  # scipy.misc.imread is deprecated in later versions

     |████████████████████████████████| 24.8MB 146kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
!git clone https://github.com/titu1994/Neural-Style-Transfer.git

Cloning into 'Neural-Style-Transfer'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1400 (delta 1), reused 3 (delta 0), pack-reused 1393
Receiving objects: 100% (1400/1400), 68.16 MiB | 41.74 MiB/s, done.
Resolving deltas: 100% (820/820), done.


In [ ]:
cd Neural-Style-Transfer/

/content/Neural-Style-Transfer


In [ ]:
pwd

'/content/Neural-Style-Transfer'

In [ ]:
# SORRY FOR MESS BUT YOU MUST REPLACE INetwork.py FILE WITH LAST CELL BEFORE RUN BELOW 

In [ ]:
LABEL_PATH   = "/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/train/labels"
IMG_PATH     = "/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/train/images"
RESULT_PATH  = "/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/style_transfer"

In [ ]:
import numpy as np
import os
from skimage.metrics import structural_similarity
import cv2
import numpy as np

In [ ]:
def find_distances():
  distances = np.zeros((20,20),dtype=float)
  for i in sorted(os.listdir(IMG_PATH)):
    for j in sorted(os.listdir(IMG_PATH)):
      a = cv2.imread(IMG_PATH + '/' + i, 0)
      b = cv2.imread(IMG_PATH + '/' + j, 0)

      score, diff = structural_similarity(a, b, full=True)
      distances[int(i.split('.')[0])][int(j.split('.')[0])] = float(score)

  return distances

In [ ]:
def style_transfer(n_of_iter = 10, content_weight = 0.025, style_weight = 1):
  distances = np.zeros((20,20))
  distances = find_distances()

  result_img = RESULT_PATH + '/images'
  result_label = RESULT_PATH + '/labels'

  os.mkdir(result_img)
  os.mkdir(result_label)

  for i in sorted(os.listdir(IMG_PATH)):
    furthest = distances[int(i.split('.')[0])].argmin()
    furthest_value = distances[int(i.split('.')[0])].min()
    os.system('python INetwork.py "'+ IMG_PATH + '/' + i +'" "'+ IMG_PATH + '/' + str(furthest) + '.png' +'" "'+ result_img + '/'+ i.split('.')[0] + '" --image_size 584 --num_iter '+ str(n_of_iter) +' --pool_type "max" --model "vgg19" --content_weight '+ str(content_weight) +' --style_weight ' + str(style_weight))
    os.rename(result_img + '/' + i.split('.')[0] + '_style.png', result_img + '/style_' + i.split('.')[0] + '.png')
    
    a = cv2.imread(IMG_PATH + '/' + i, 0)
    b = cv2.imread(result_img + '/style_' + i , 0)
    score, diff = structural_similarity(a, b, full=True)
    print("style_" + i + " saved successfully.\nFurthest image from " + i + " is " + str(furthest) + " with similarity: " + str(furthest_value) + "\nSimilarity between " + i + " and styled one is: " + str(score) + "\n---------------------------------------------------------------------") 

  os.system("rsync -a "+ LABEL_PATH + "/ " + result_label)
  for i in sorted(os.listdir(result_label)):
    os.rename(result_label + '/' + i, result_label + '/style_' + i.split('.')[0] + '.png')
  print('Label folder successfully copied from LABEL_PATH.')


In [ ]:
style_transfer(n_of_iter = 10, content_weight = 0.025, style_weight = 1)

style_0.png saved successfully.
Furthest image from 0.png is 13 with similarity: 0.6581827873570326
Similarity between 0.png and styled one is: 0.916427185561389
---------------------------------------------------------------------
style_1.png saved successfully.
Furthest image from 1.png is 13 with similarity: 0.6545721609734262
Similarity between 1.png and styled one is: 0.9423020593559593
---------------------------------------------------------------------
style_10.png saved successfully.
Furthest image from 10.png is 13 with similarity: 0.6619912221219528
Similarity between 10.png and styled one is: 0.9374854737404626
---------------------------------------------------------------------
style_11.png saved successfully.
Furthest image from 11.png is 13 with similarity: 0.6916511963871375
Similarity between 11.png and styled one is: 0.9240754506464935
---------------------------------------------------------------------
style_12.png saved successfully.
Furthest image from 12.png is 

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

# from scipy.misc import imread, imresize, imsave, fromimage, toimage
from utils import imread, imresize, imsave, fromimage, toimage

from scipy.optimize import fmin_l_bfgs_b
import numpy as np
import time
import argparse
import warnings

from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, AveragePooling2D, MaxPooling2D
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils.layer_utils import convert_all_kernels_in_model

"""
Neural Style Transfer with Keras 2.0.5

Based on:
https://github.com/fchollet/keras/blob/master/examples/neural_style_transfer.py

Contains few improvements suggested in the paper Improving the Neural Algorithm of Artistic Style
(http://arxiv.org/abs/1605.04603).

-----------------------------------------------------------------------------------------------------------------------
"""

THEANO_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_th_dim_ordering_th_kernels_notop.h5'
TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

TH_19_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_th_dim_ordering_th_kernels_notop.h5'
TF_19_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'


parser = argparse.ArgumentParser(description='Neural style transfer with Keras.')
parser.add_argument('base_image_path', metavar='base', type=str,
                    help='Path to the image to transform.')

parser.add_argument('syle_image_paths', metavar='ref', nargs='+', type=str,
                    help='Path to the style reference image.')

parser.add_argument('result_prefix', metavar='res_prefix', type=str,
                    help='Prefix for the saved results.')

parser.add_argument("--style_masks", type=str, default=None, nargs='+',
                    help='Masks for style images')

parser.add_argument("--content_mask", type=str, default=None,
                    help='Masks for the content image')

parser.add_argument("--color_mask", type=str, default=None,
                    help='Mask for color preservation')

parser.add_argument("--image_size", dest="img_size", default=400, type=int,
                    help='Minimum image size')

parser.add_argument("--content_weight", dest="content_weight", default=0.025, type=float,
                    help="Weight of content")

parser.add_argument("--style_weight", dest="style_weight", nargs='+', default=[1], type=float,
                    help="Weight of style, can be multiple for multiple styles")

parser.add_argument("--style_scale", dest="style_scale", default=1.0, type=float,
                    help="Scale the weighing of the style")

parser.add_argument("--total_variation_weight", dest="tv_weight", default=8.5e-5, type=float,
                    help="Total Variation weight")

parser.add_argument("--num_iter", dest="num_iter", default=10, type=int,
                    help="Number of iterations")

parser.add_argument("--model", default="vgg16", type=str,
                    help="Choices are 'vgg16' and 'vgg19'")

parser.add_argument("--content_loss_type", default=0, type=int,
                    help='Can be one of 0, 1 or 2. Readme contains the required information of each mode.')

parser.add_argument("--rescale_image", dest="rescale_image", default="False", type=str,
                    help="Rescale image after execution to original dimentions")

parser.add_argument("--rescale_method", dest="rescale_method", default="bilinear", type=str,
                    help="Rescale image algorithm")

parser.add_argument("--maintain_aspect_ratio", dest="maintain_aspect_ratio", default="True", type=str,
                    help="Maintain aspect ratio of loaded images")

parser.add_argument("--content_layer", dest="content_layer", default="conv5_2", type=str,
                    help="Content layer used for content loss.")

parser.add_argument("--init_image", dest="init_image", default="content", type=str,
                    help="Initial image used to generate the final image. Options are 'content', 'noise', or 'gray'")

parser.add_argument("--pool_type", dest="pool", default="max", type=str,
                    help='Pooling type. Can be "ave" for average pooling or "max" for max pooling')

parser.add_argument('--preserve_color', dest='color', default="False", type=str,
                    help='Preserve original color in image')

parser.add_argument('--min_improvement', default=0.0, type=float,
                    help='Defines minimum improvement required to continue script')


def str_to_bool(v):
    return v.lower() in ("true", "yes", "t", "1")

''' Arguments '''

args = parser.parse_args()
base_image_path = args.base_image_path
style_reference_image_paths = args.syle_image_paths
result_prefix = args.result_prefix

style_image_paths = []
for style_image_path in style_reference_image_paths:
    style_image_paths.append(style_image_path)

style_masks_present = args.style_masks is not None
mask_paths = []

if style_masks_present:
    for mask_path in args.style_masks:
        mask_paths.append(mask_path)

if style_masks_present:
    assert len(style_image_paths) == len(mask_paths), "Wrong number of style masks provided.\n" \
                                                      "Number of style images = %d, \n" \
                                                      "Number of style mask paths = %d." % \
                                                      (len(style_image_paths), len(style_masks_present))

content_mask_present = args.content_mask is not None
content_mask_path = args.content_mask


color_mask_present = args.color_mask is not None

rescale_image = str_to_bool(args.rescale_image)
maintain_aspect_ratio = str_to_bool(args.maintain_aspect_ratio)
preserve_color = str_to_bool(args.color)

# these are the weights of the different loss components
content_weight = args.content_weight
total_variation_weight = args.tv_weight

style_weights = []

if len(style_image_paths) != len(args.style_weight):
    print("Mismatch in number of style images provided and number of style weights provided. \n"
          "Found %d style images and %d style weights. \n"
          "Equally distributing weights to all other styles." % (len(style_image_paths), len(args.style_weight)))

    weight_sum = sum(args.style_weight) * args.style_scale
    count = len(style_image_paths)

    for i in range(len(style_image_paths)):
        style_weights.append(weight_sum / count)
else:
    for style_weight in args.style_weight:
        style_weights.append(style_weight * args.style_scale)

# Decide pooling function
pooltype = str(args.pool).lower()
assert pooltype in ["ave", "max"], 'Pooling argument is wrong. Needs to be either "ave" or "max".'

pooltype = 1 if pooltype == "ave" else 0

read_mode = "gray" if args.init_image == "gray" else "color"

# dimensions of the generated picture.
img_width = img_height = 0

img_WIDTH = img_HEIGHT = 0
aspect_ratio = 0

assert args.content_loss_type in [0, 1, 2], "Content Loss Type must be one of 0, 1 or 2"


# util function to open, resize and format pictures into appropriate tensors
def preprocess_image(image_path, load_dims=False, read_mode="color"):
    global img_width, img_height, img_WIDTH, img_HEIGHT, aspect_ratio

    mode = "RGB" if read_mode == "color" else "L"
    img = imread(image_path, mode=mode)  # Prevents crashes due to PNG images (ARGB)

    if mode == "L":
        # Expand the 1 channel grayscale to 3 channel grayscale image
        temp = np.zeros(img.shape + (3,), dtype=np.uint8)
        temp[:, :, 0] = img
        temp[:, :, 1] = img.copy()
        temp[:, :, 2] = img.copy()

        img = temp

    if load_dims:
        img_WIDTH = img.shape[0]
        img_HEIGHT = img.shape[1]
        aspect_ratio = float(img_HEIGHT) / img_WIDTH

        img_width = args.img_size
        if maintain_aspect_ratio:
            img_height = int(img_width * aspect_ratio)
        else:
            img_height = args.img_size

    img = imresize(img, (img_width, img_height)).astype('float32')

    # RGB -> BGR
    img = img[:, :, ::-1]

    img[:, :, 0] -= 103.939
    img[:, :, 1] -= 116.779
    img[:, :, 2] -= 123.68

    if K.image_data_format() == "channels_first":
        img = img.transpose((2, 0, 1)).astype('float32')

    img = np.expand_dims(img, axis=0)
    return img


# util function to convert a tensor into a valid image
def deprocess_image(x):
    if K.image_data_format() == "channels_first":
        x = x.reshape((3, img_width, img_height))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_width, img_height, 3))

    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # BGR -> RGB
    x = x[:, :, ::-1]

    x = np.clip(x, 0, 255).astype('uint8')
    return x


# util function to preserve image color
def original_color_transform(content, generated, mask=None):
    generated = fromimage(toimage(generated, mode='RGB'), mode='YCbCr')  # Convert to YCbCr color space

    if mask is None:
        generated[:, :, 1:] = content[:, :, 1:]  # Generated CbCr = Content CbCr
    else:
        width, height, channels = generated.shape

        for i in range(width):
            for j in range(height):
                if mask[i, j] == 1:
                    generated[i, j, 1:] = content[i, j, 1:]

    generated = fromimage(toimage(generated, mode='YCbCr'), mode='RGB')  # Convert to RGB color space
    return generated


def load_mask(mask_path, shape, return_mask_img=False):
    if K.image_data_format() == "channels_first":
        _, channels, width, height = shape
    else:
        _, width, height, channels = shape

    mask = imread(mask_path, mode="L") # Grayscale mask load
    mask = imresize(mask, (width, height)).astype('float32')

    # Perform binarization of mask
    mask[mask <= 127] = 0
    mask[mask > 128] = 255

    max = np.amax(mask)
    mask /= max

    if return_mask_img: return mask

    mask_shape = shape[1:]

    mask_tensor = np.empty(mask_shape)

    for i in range(channels):
        if K.image_data_format() == "channels_first":
            mask_tensor[i, :, :] = mask
        else:
            mask_tensor[:, :, i] = mask

    return mask_tensor


def pooling_func(x):
    if pooltype == 1:
        return AveragePooling2D((2, 2), strides=(2, 2))(x)
    else:
        return MaxPooling2D((2, 2), strides=(2, 2))(x)


# get tensor representations of our images
base_image = K.variable(preprocess_image(base_image_path, True, read_mode=read_mode))

style_reference_images = []
for style_path in style_image_paths:
    style_reference_images.append(K.variable(preprocess_image(style_path)))

# this will contain our generated image
if K.image_data_format() == "channels_first":
    combination_image = K.placeholder((1, 3, img_width, img_height))
else:
    combination_image = K.placeholder((1, img_width, img_height, 3))

image_tensors = [base_image]
for style_image_tensor in style_reference_images:
    image_tensors.append(style_image_tensor)
image_tensors.append(combination_image)

nb_tensors = len(image_tensors)
nb_style_images = nb_tensors - 2 # Content and Output image not considered

# combine the various images into a single Keras tensor
input_tensor = K.concatenate(image_tensors, axis=0)

if K.image_data_format() == "channels_first":
    shape = (nb_tensors, 3, img_width, img_height)
else:
    shape = (nb_tensors, img_width, img_height, 3)

ip = Input(tensor=input_tensor, batch_shape=shape)

# build the VGG16 network with our 3 images as input
x = Convolution2D(64, (3, 3), activation='relu', name='conv1_1', padding='same')(ip)
x = Convolution2D(64, (3, 3), activation='relu', name='conv1_2', padding='same')(x)
x = pooling_func(x)

x = Convolution2D(128, (3, 3), activation='relu', name='conv2_1', padding='same')(x)
x = Convolution2D(128, (3, 3), activation='relu', name='conv2_2', padding='same')(x)
x = pooling_func(x)

x = Convolution2D(256, (3, 3), activation='relu', name='conv3_1', padding='same')(x)
x = Convolution2D(256, (3, 3), activation='relu', name='conv3_2', padding='same')(x)
x = Convolution2D(256, (3, 3), activation='relu', name='conv3_3', padding='same')(x)
if args.model == "vgg19":
    x = Convolution2D(256, (3, 3), activation='relu', name='conv3_4', padding='same')(x)
x = pooling_func(x)

x = Convolution2D(512, (3, 3), activation='relu', name='conv4_1', padding='same')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv4_2', padding='same')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv4_3', padding='same')(x)
if args.model == "vgg19":
    x = Convolution2D(512, (3, 3), activation='relu', name='conv4_4', padding='same')(x)
x = pooling_func(x)

x = Convolution2D(512, (3, 3), activation='relu', name='conv5_1', padding='same')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv5_2', padding='same')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv5_3', padding='same')(x)
if args.model == "vgg19":
    x = Convolution2D(512, (3, 3), activation='relu', name='conv5_4', padding='same')(x)
x = pooling_func(x)

model = Model(ip, x)

if K.image_data_format() == "channels_first":
    if args.model == "vgg19":
        weights = get_file('vgg19_weights_th_dim_ordering_th_kernels_notop.h5', TH_19_WEIGHTS_PATH_NO_TOP, cache_subdir='models')
    else:
        weights = get_file('vgg16_weights_th_dim_ordering_th_kernels_notop.h5', THEANO_WEIGHTS_PATH_NO_TOP, cache_subdir='models')
else:
    if args.model == "vgg19":
        weights = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', TF_19_WEIGHTS_PATH_NO_TOP, cache_subdir='models')
    else:
        weights = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', TF_WEIGHTS_PATH_NO_TOP, cache_subdir='models')

model.load_weights(weights)

if K.backend() == 'tensorflow' and K.image_data_format() == "channels_first":
    warnings.warn('You are using the TensorFlow backend, yet you '
                  'are using the Theano '
                  'image dimension ordering convention '
                  '(`image_dim_ordering="th"`). '
                  'For best performance, set '
                  '`image_dim_ordering="tf"` in '
                  'your Keras config '
                  'at ~/.keras/keras.json.')
    convert_all_kernels_in_model(model)

print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
shape_dict = dict([(layer.name, layer.output_shape) for layer in model.layers])

# compute the neural style loss
# first we need to define 4 util functions

# Improvement 1
# the gram matrix of an image tensor (feature-wise outer product) using shifted activations
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == "channels_first":
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features - 1, K.transpose(features - 1))
    return gram


# the "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image
def style_loss(style, combination, mask_path=None, nb_channels=None):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3

    if content_mask_path is not None:
        content_mask = K.variable(load_mask(content_mask_path, nb_channels))
        combination = combination * K.stop_gradient(content_mask)
        del content_mask

    if mask_path is not None:
        style_mask = K.variable(load_mask(mask_path, nb_channels))
        style = style * K.stop_gradient(style_mask)
        if content_mask_path is None:
            combination = combination * K.stop_gradient(style_mask)
        del style_mask

    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_width * img_height
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):
    channel_dim = 0 if K.image_data_format() == "channels_first" else -1

    try:
        channels = K.int_shape(base)[channel_dim]
    except TypeError:
        channels = K.shape(base)[channel_dim]
    size = img_width * img_height

    if args.content_loss_type == 1:
        multiplier = 1. / (2. * (channels ** 0.5) * (size ** 0.5))
    elif args.content_loss_type == 2:
        multiplier = 1. / (channels * size)
    else:
        multiplier = 1.

    return multiplier * K.sum(K.square(combination - base))


# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == "channels_first":
        a = K.square(x[:, :, :img_width - 1, :img_height - 1] - x[:, :, 1:, :img_height - 1])
        b = K.square(x[:, :, :img_width - 1, :img_height - 1] - x[:, :, :img_width - 1, 1:])
    else:
        a = K.square(x[:, :img_width - 1, :img_height - 1, :] - x[:, 1:, :img_height - 1, :])
        b = K.square(x[:, :img_width - 1, :img_height - 1, :] - x[:, :img_width - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

if args.model == "vgg19":
    feature_layers = ['conv1_1', 'conv1_2', 'conv2_1', 'conv2_2', 'conv3_1', 'conv3_2', 'conv3_3', 'conv3_4',
                      'conv4_1', 'conv4_2', 'conv4_3', 'conv4_4', 'conv5_1', 'conv5_2', 'conv5_3', 'conv5_4']
else:
    feature_layers = ['conv1_1', 'conv1_2', 'conv2_1', 'conv2_2', 'conv3_1', 'conv3_2', 'conv3_3',
                      'conv4_1', 'conv4_2', 'conv4_3', 'conv5_1', 'conv5_2', 'conv5_3']

# combine these loss functions into a single scalar
loss = K.variable(0.)
layer_features = outputs_dict[args.content_layer]
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[nb_tensors - 1, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                      combination_features)
# Improvement 2
# Use all layers for style feature extraction and reconstruction
nb_layers = len(feature_layers) - 1

style_masks = []
if style_masks_present:
    style_masks = mask_paths # If mask present, pass dictionary of masks to style loss
else:
    style_masks = [None for _ in range(nb_style_images)] # If masks not present, pass None to the style loss

channel_index = 1 if K.image_data_format() == "channels_first" else -1

# Improvement 3 : Chained Inference without blurring
for i in range(len(feature_layers) - 1):
    layer_features = outputs_dict[feature_layers[i]]
    shape = shape_dict[feature_layers[i]]
    combination_features = layer_features[nb_tensors - 1, :, :, :]
    style_reference_features = layer_features[1:nb_tensors - 1, :, :, :]
    sl1 = []
    for j in range(nb_style_images):
        sl1.append(style_loss(style_reference_features[j], combination_features, style_masks[j], shape))

    layer_features = outputs_dict[feature_layers[i + 1]]
    shape = shape_dict[feature_layers[i + 1]]
    combination_features = layer_features[nb_tensors - 1, :, :, :]
    style_reference_features = layer_features[1:nb_tensors - 1, :, :, :]
    sl2 = []
    for j in range(nb_style_images):
        sl2.append(style_loss(style_reference_features[j], combination_features, style_masks[j], shape))

    for j in range(nb_style_images):
        sl = sl1[j] - sl2[j]

        # Improvement 4
        # Geometric weighted scaling of style loss
        loss = loss + (style_weights[j] / (2 ** (nb_layers - (i + 1)))) * sl

loss = loss + total_variation_weight * total_variation_loss(combination_image)

# get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)

outputs = [loss]
if type(grads) in {list, tuple}:
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)


def eval_loss_and_grads(x):
    if K.image_data_format() == "channels_first":
        x = x.reshape((1, 3, img_width, img_height))
    else:
        x = x.reshape((1, img_width, img_height, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values


# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss


if "content" in args.init_image or "gray" in args.init_image:
    x = preprocess_image(base_image_path, True, read_mode=read_mode)
elif "noise" in args.init_image:
    x = np.random.uniform(0, 255, (1, img_width, img_height, 3)) - 128.

    if K.image_data_format() == "channels_first":
        x = x.transpose((0, 3, 1, 2))
else:
    print("Using initial image : ", args.init_image)
    x = preprocess_image(args.init_image, read_mode=read_mode)

# We require original image if we are to preserve color in YCbCr mode
if preserve_color:
    content = imread(base_image_path, mode="YCbCr")
    content = imresize(content, (img_width, img_height))

    if color_mask_present:
        if K.image_data_format() == "channels_first":
            color_mask_shape = (None, None, img_width, img_height)
        else:
            color_mask_shape = (None, img_width, img_height, None)

        color_mask = load_mask(args.color_mask, color_mask_shape, return_mask_img=True)
    else:
        color_mask = None
else:
    color_mask = None

num_iter = args.num_iter
prev_min_val = -1

improvement_threshold = float(args.min_improvement)

for i in range(num_iter):
    print("Starting iteration %d of %d" % ((i + 1), num_iter))
    start_time = time.time()

    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)

    if prev_min_val == -1:
        prev_min_val = min_val

    improvement = (prev_min_val - min_val) / prev_min_val * 100

    print("Current loss value:", min_val, " Improvement : %0.3f" % improvement, "%")
    prev_min_val = min_val
    # save current generated image
    img = deprocess_image(x.copy())

    if preserve_color and content is not None:
        img = original_color_transform(content, img, mask=color_mask)

    if not rescale_image:
        img_ht = int(img_width * aspect_ratio)
        print("Rescaling Image to (%d, %d)" % (img_width, img_ht))
        img = imresize(img, (img_width, img_ht), interp=args.rescale_method)

    if rescale_image:
        print("Rescaling Image to (%d, %d)" % (img_WIDTH, img_HEIGHT))
        img = imresize(img, (img_WIDTH, img_HEIGHT), interp=args.rescale_method)
    if i == num_iter-1:
      fname = result_prefix + "_style.png"
      imsave(fname, img)
      print("Image saved as", fname)
    end_time = time.time()
    
    print("Iteration %d completed in %ds" % (i + 1, end_time - start_time))

    if improvement_threshold is not 0.0:
        if improvement < improvement_threshold and improvement is not 0.0:
            print("Improvement (%f) is less than improvement threshold (%f). Early stopping script." %
                  (improvement, improvement_threshold))
            exit()

In [ ]:
#A
len(['train', 
                     'flipping0', 
                     'flipping1', 
                     'rotation30', 
                     'rotation60', 
                     'rotation90', 
                     'rotation120', 
                     'rotation150', 
                     'rotation180', 
                     'rotation210', 
                     'rotation240', 
                     'rotation270', 
                     'rotation300', 
                     'rotation330', 
                     'flipping0_rotat30','flipping0_rotat60',
                     'flipping0_rotat90','flipping0_rotat120',
                     'flipping0_rotat150',
                     'flipping0_rotat180',
                     'flipping0_rotat210','flipping0_rotat240',
                     'flipping0_rotat270',
                     'flipping0_rotat300','flipping0_rotat330',
                     'flipping1_rotat30','flipping1_rotat60',
                     'flipping1_rotat90','flipping1_rotat120',
                     'flipping1_rotat150',
                     'flipping1_rotat180',
                     'flipping1_rotat210','flipping1_rotat240',
                     'flipping1_rotat270',
                     'flipping1_rotat300','flipping1_rotat330',
                     'zoom_out08',
                     'zoom_out08_rotat30', 'zoom_out08_rotat60',
                     'zoom_out08_rotat90',
                     'zoom_out08_rotat120',
                     'zoom_out08_rotat180','zoom_out08_rotat150',
                     'zoom_out08_rotat210','zoom_out08_rotat240',
                     'zoom_out08_rotat270',
                     'zoom_out08_rotat300',
                     'zoom_out08_rotat330',
                     'white_noise10',
                     'elastic',
                     'shifty_100','shifty_-100','shiftx_100','shiftx_-100',
                     'shifty_100_90','shifty_100_180','shifty_100_270',
                     'shifty_-100_90','shifty_-100_180','shifty_-100_270',
                     'shiftx_100_90','shiftx_100_180','shiftx_100_270',
                     'shiftx_-100_90','shiftx_-100_180','shiftx_-100_270'
                     ])


A_gamma_album = ['train', 
                     'flipping0', 
                     'flipping1', 
                     'rotation30', 
                     'rotation60', 
                     'rotation90', 
                     'rotation120', 
                     'rotation150', 
                     'rotation180', 
                     'rotation210', 
                     'rotation240', 
                     'rotation270', 
                     'rotation300', 
                     'rotation330', 
                     'flipping0_rotat30','flipping0_rotat60',
                     'flipping0_rotat90','flipping0_rotat120',
                     'flipping0_rotat150',
                     'flipping0_rotat180',
                     'flipping0_rotat210','flipping0_rotat240',
                     'flipping0_rotat270',
                     'flipping0_rotat300','flipping0_rotat330',
                     'flipping1_rotat30','flipping1_rotat60',
                     'flipping1_rotat90','flipping1_rotat120',
                     'flipping1_rotat150',
                     'flipping1_rotat180',
                     'flipping1_rotat210','flipping1_rotat240',
                     'flipping1_rotat270',
                     'flipping1_rotat300','flipping1_rotat330',
                     'zoom_out08',
                     'zoom_out08_rotat30', 'zoom_out08_rotat60',
                     'zoom_out08_rotat90',
                     'zoom_out08_rotat120',
                     'zoom_out08_rotat180','zoom_out08_rotat150',
                     'zoom_out08_rotat210','zoom_out08_rotat240',
                     'zoom_out08_rotat270',
                     'zoom_out08_rotat300',
                     'zoom_out08_rotat330',
                     'white_noise10',
                     'elastic',
                     'shifty_100','shifty_-100','shiftx_100','shiftx_-100',
                     'shifty_100_90','shifty_100_180','shifty_100_270',
                     'shifty_-100_90','shifty_-100_180','shifty_-100_270',
                     'shiftx_100_90','shiftx_100_180','shiftx_100_270',
                     'shiftx_-100_90','shiftx_-100_180','shiftx_-100_270',
                     'gamma_corr05','gamma_corr05_rotation30','gamma_corr05_rotation60',
                     'gamma_corr05_rotation90','gamma_corr05_rotation120','gamma_corr05_rotation150',
                     'gamma_corr05_rotation180','gamma_corr05_rotation210','gamma_corr05_rotation240',
                     'gamma_corr05_rotation270','gamma_corr05_rotation300','gamma_corr05_rotation330',
                     'album_elastic','album_grid_dist','album_heavy', 'album_medium', 'album_optical','album_random_crop']

B: ['train', 
                     'flipping0', 
                     'flipping1', 
                     'rotation30', 
                     'rotation60', 
                     'rotation90', 
                     'rotation120', 
                     'rotation150', 
                     'rotation180', 
                     'rotation210', 
                     'rotation240', 
                     'rotation270', 
                     'rotation300', 
                     'rotation330', 
                     'flipping0_rotat30','flipping0_rotat60',
                     'flipping0_rotat90','flipping0_rotat120',
                     'flipping0_rotat150',
                     'flipping0_rotat180',
                     'flipping0_rotat210','flipping0_rotat240',
                     'flipping0_rotat270',
                     'flipping0_rotat300','flipping0_rotat330',
                     'flipping1_rotat30','flipping1_rotat60',
                     'flipping1_rotat90','flipping1_rotat120',
                     'flipping1_rotat150',
                     'flipping1_rotat180',
                     'flipping1_rotat210','flipping1_rotat240',
                     'flipping1_rotat270',
                     'flipping1_rotat300','flipping1_rotat330',
                     'zoom_out08',
                     'zoom_out08_rotat30', 'zoom_out08_rotat60',
                     'zoom_out08_rotat90',
                     'zoom_out08_rotat120',
                     'zoom_out08_rotat180','zoom_out08_rotat150',
                     'zoom_out08_rotat210','zoom_out08_rotat240',
                     'zoom_out08_rotat270',
                     'zoom_out08_rotat300',
                     'zoom_out08_rotat330',
                     'white_noise10','white_noise10_90','white_noise10_180','white_noise10_270',
                     'elastic',
                     'shifty_100','shifty_-100','shiftx_100','shiftx_-100',
                     'shifty_100_90','shifty_100_180','shifty_100_270',
                     'shifty_-100_90','shifty_-100_180','shifty_-100_270',
                     'shiftx_100_90','shiftx_100_180','shiftx_100_270',
                     'shiftx_-100_90','shiftx_-100_180','shiftx_-100_270',
                     'gamma_corr05','gamma_corr05_rotation30','gamma_corr05_rotation60',
                     'gamma_corr05_rotation90','gamma_corr05_rotation120','gamma_corr05_rotation150',
                     'gamma_corr05_rotation180','gamma_corr05_rotation210','gamma_corr05_rotation240',
                     'gamma_corr05_rotation270','gamma_corr05_rotation300','gamma_corr05_rotation330',
                     'album_elastic','album_grid_dist','album_heavy', 'album_medium', 'album_optical','album_random_crop',
                     'album_elastic90', 'album_elastic180', 'album_grid_dist90','album_grid_dist180','album_grid_dist270',
                     'album_heavy90','album_heavy180','album_heavy270','album_medium90','album_medium180','album_medium270',
                     'album_optical90', 'album_optical180', 'album_optical270','album_random_crop2','album_random_crop3','album_random_crop4',
                     'blurring3', 'blurring5','dropout010','eq_hist', 'sharpen']